Proposal for Group 42

Let's upload our possible datasets in the data folder so we can easily have a look at them and possibly run some functions on them if we need. 

In [16]:
library(tidyverse)


In [40]:
heart_data <- read_csv("data/heart_failure_clinical_records_dataset.csv")%>%
    mutate(sex = as_factor(sex))%>%
    mutate(smoking = as_factor(smoking))%>%
    mutate(DEATH_EVENT = as_factor(DEATH_EVENT))%>%
    mutate(high_blood_pressure = as_factor(high_blood_pressure))%>%
    mutate(diabetes = as_factor(diabetes))%>%
    mutate(anaemia<- as_factor(anaemia))
    
levels(heart_data$sex)<- c("female","male")
levels(heart_data$smoking)<- c("no","yes")
levels(heart_data$DEATH_EVENT)<- c("no","yes")
levels(heart_data$high_blood_pressure)<- c("no","yes")
levels(heart_data$diabetes)<- c("no","yes")
levels(heart_data$anaemia)<- c("no","yes")

heart_data

Parsed with column specification:
cols(
  age = col_double(),
  anaemia = col_double(),
  creatinine_phosphokinase = col_double(),
  diabetes = col_double(),
  ejection_fraction = col_double(),
  high_blood_pressure = col_double(),
  platelets = col_double(),
  serum_creatinine = col_double(),
  serum_sodium = col_double(),
  sex = col_double(),
  smoking = col_double(),
  time = col_double(),
  DEATH_EVENT = col_double()
)



age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT,anaemia <- as_factor(anaemia)
<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<fct>
75,0,582,no,20,yes,265000,1.90,130,male,no,4,yes,0
55,0,7861,no,38,no,263358,1.10,136,male,no,6,yes,0
65,0,146,no,20,no,162000,1.30,129,male,yes,7,yes,0
50,1,111,no,20,no,210000,1.90,137,male,no,7,yes,1
65,1,160,yes,20,no,327000,2.70,116,female,no,8,yes,1
90,1,47,no,40,yes,204000,2.10,132,male,yes,8,yes,1
75,1,246,no,15,no,127000,1.20,137,male,no,10,yes,1
60,1,315,yes,60,no,454000,1.10,131,male,yes,10,yes,1
65,0,157,no,65,no,263358,1.50,138,female,no,10,yes,0
